In [3]:
# Dependencies and Setup
# !pip install citipy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import re
from pprint import pprint
from sqlalchemy import create_engine

In [5]:
csv_file = "../Resources/branded_food.csv"
branded_food_df = pd.read_csv(csv_file)
branded_food_df.head()

C:\Users\17062\anaconda\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,fdc_id,brand_owner,gtin_upc,ingredients,serving_size,serving_size_unit,household_serving_fulltext,branded_food_category,data_source,modified_date,available_date
0,356425,"G. T. Japan, Inc.",1.90221e+10,"ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR...",40.0,g,1 PIECE,Ice Cream & Frozen Yogurt,LI,11/15/2017,11/15/2017
1,356426,FRESH & EASY,5.05e+12,"WATER, SUGAR, TOMATO PASTE, MOLASSES, DISTILLE...",37.0,g,2 Tbsp,"Ketchup, Mustard, BBQ & Cheese Sauce",LI,4/26/2018,4/26/2018
2,356427,FRESH & EASY,5.05e+12,"SUGAR, WATER, DISTILLED VINEGAR, TOMATO PASTE,...",34.0,g,2 Tbsp,"Ketchup, Mustard, BBQ & Cheese Sauce",LI,4/26/2018,4/26/2018
3,356428,FRESH & EASY,5.05e+12,"TOMATO PUREE (WATER, TOMATO PASTE), SUGAR, DIS...",35.0,g,2 Tbsp,"Ketchup, Mustard, BBQ & Cheese Sauce",LI,4/26/2018,4/26/2018
4,356429,FRESH & EASY,5.05e+12,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PASTE,...",37.0,g,2 Tbsp,"Ketchup, Mustard, BBQ & Cheese Sauce",LI,4/26/2018,4/26/2018


In [6]:
clean_branded_food_df = branded_food_df[['fdc_id', 'brand_owner', 'ingredients', 'branded_food_category']].copy()
clean_branded_food_df.head()

,fdc_id,brand_owner,ingredients,branded_food_category
0,356425,"G. T. Japan, Inc.","ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR...",Ice Cream & Frozen Yogurt
1,356426,FRESH & EASY,"WATER, SUGAR, TOMATO PASTE, MOLASSES, DISTILLE...","Ketchup, Mustard, BBQ & Cheese Sauce"
2,356427,FRESH & EASY,"SUGAR, WATER, DISTILLED VINEGAR, TOMATO PASTE,...","Ketchup, Mustard, BBQ & Cheese Sauce"
3,356428,FRESH & EASY,"TOMATO PUREE (WATER, TOMATO PASTE), SUGAR, DIS...","Ketchup, Mustard, BBQ & Cheese Sauce"
4,356429,FRESH & EASY,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PASTE,...","Ketchup, Mustard, BBQ & Cheese Sauce"


In [7]:
clean_branded_food_df_500 = clean_branded_food_df.head(500)

In [8]:
clean_branded_food_df_500.head(20)

,fdc_id,brand_owner,ingredients,branded_food_category
0,356425,"G. T. Japan, Inc.","ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR...",Ice Cream & Frozen Yogurt
1,356426,FRESH & EASY,"WATER, SUGAR, TOMATO PASTE, MOLASSES, DISTILLE...","Ketchup, Mustard, BBQ & Cheese Sauce"
2,356427,FRESH & EASY,"SUGAR, WATER, DISTILLED VINEGAR, TOMATO PASTE,...","Ketchup, Mustard, BBQ & Cheese Sauce"
3,356428,FRESH & EASY,"TOMATO PUREE (WATER, TOMATO PASTE), SUGAR, DIS...","Ketchup, Mustard, BBQ & Cheese Sauce"
4,356429,FRESH & EASY,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PASTE,...","Ketchup, Mustard, BBQ & Cheese Sauce"
5,356430,FRESH & EASY,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PASTE,...","Ketchup, Mustard, BBQ & Cheese Sauce"
6,356431,FRESH & EASY,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PUREE ...","Ketchup, Mustard, BBQ & Cheese Sauce"
7,356432,FRESH & EASY,"WATER, SUGAR, TOMATO PASTE, MOLASSES, DISTILLE...","Ketchup, Mustard, BBQ & Cheese Sauce"
8,356433,FRESH & EASY,"TOMATO PUREE, ONION PUREE, SUGAR, MOLASSES, DI...","Ketchup, Mustard, BBQ & Cheese Sauce"
9,356434,FRESH & EASY,"ORGANIC GRAIN VINEGAR, WATER, ORGANIC MUSTARD ...","Ketchup, Mustard, BBQ & Cheese Sauce"


In [9]:
# create list for ingredients

ingred_df = clean_branded_food_df_500[["fdc_id","ingredients"]]
ingred_df.head()

,fdc_id,ingredients
0,356425,"ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR..."
1,356426,"WATER, SUGAR, TOMATO PASTE, MOLASSES, DISTILLE..."
2,356427,"SUGAR, WATER, DISTILLED VINEGAR, TOMATO PASTE,..."
3,356428,"TOMATO PUREE (WATER, TOMATO PASTE), SUGAR, DIS..."
4,356429,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PASTE,..."


In [10]:
ingred_df_ingredients =ingred_df["ingredients"]

In [11]:
ingred_df_ingredients.dtype

dtype('O')

In [12]:
ingred_df_ingredients.str.split()

0      [ICE, CREAM, INGREDIENTS:, MILK,, CREAM,, SUGA...
1      [WATER,, SUGAR,, TOMATO, PASTE,, MOLASSES,, DI...
2      [SUGAR,, WATER,, DISTILLED, VINEGAR,, TOMATO, ...
3      [TOMATO, PUREE, (WATER,, TOMATO, PASTE),, SUGA...
4      [SUGAR,, DISTILLED, VINEGAR,, WATER,, TOMATO, ...
5      [SUGAR,, DISTILLED, VINEGAR,, WATER,, TOMATO, ...
6      [SUGAR,, DISTILLED, VINEGAR,, WATER,, TOMATO, ...
7      [WATER,, SUGAR,, TOMATO, PASTE,, MOLASSES,, DI...
8      [TOMATO, PUREE,, ONION, PUREE,, SUGAR,, MOLASS...
9      [ORGANIC, GRAIN, VINEGAR,, WATER,, ORGANIC, MU...
10     [DISTILLED, VINEGAR,, WATER,, HONEY,, MUSTARD,...
11     [DISTILLED, VINEGAR,, WATER,, MUSTARD, SEED,, ...
12     [ORGANIC, DISTILLED, VINEGAR,, WATER,, ORGANIC...
13     [DISTILLED, VINEGAR,, WATER,, MUSTARD, SEED,, ...
14     [DISTILLED, VINEGAR,, WATER,, MUSTARD, SEED,, ...
15     [DISTILLED, VINEGAR,, WATER,, MUSTARD, SEED,, ...
16     [ORGANIC, TOMATO, CONCENTRATE, (WATER,, ORGANI...
17     [WATER,, SOYBEAN, OIL,, 

In [13]:
print(type(ingred_df))

<class 'pandas.core.frame.DataFrame'>


In [14]:
#Drop null values
ingred_df.dropna()
ingred_df.count()

fdc_id         500
ingredients    500
dtype: int64

In [17]:
# use itertuples to scan dataframe for regex pattern ORGANIC
organic_count = 0
search_word = re.compile("\W*(ORGANIC)\W*")
ingredient_count = 0
counter=0
organic_list = []
# for each row in the dataframe
# itertuples will return each row as row.header_title
ingredient_count2 = []
organic_count2 = []
for row in ingred_df.itertuples():
    #print(row)
    ingredient_count = len(row.ingredients.split(",")) #ingredients is a bunch of strings, convert to list and count
    organic_list = re.findall(search_word,row.ingredients)
    organic_count = len(organic_list)
    #print(ingred_df.head())
    key = row.fdc_id
    ingredient_count2.append(ingredient_count)
    organic_count2.append(organic_count)

    counter += 1
    
ingred_df["ingredient_count"] = ingredient_count2
ingred_df["organic_count"] = organic_count2

In [18]:
ingred_df.head(20)

,fdc_id,ingredients,ingredient_count,organic_count
0,356425,"ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR...",21,0
1,356426,"WATER, SUGAR, TOMATO PASTE, MOLASSES, DISTILLE...",13,0
2,356427,"SUGAR, WATER, DISTILLED VINEGAR, TOMATO PASTE,...",15,0
3,356428,"TOMATO PUREE (WATER, TOMATO PASTE), SUGAR, DIS...",16,0
4,356429,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PASTE,...",15,0
5,356430,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PASTE,...",14,0
6,356431,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PUREE ...",15,0
7,356432,"WATER, SUGAR, TOMATO PASTE, MOLASSES, DISTILLE...",15,0
8,356433,"TOMATO PUREE, ONION PUREE, SUGAR, MOLASSES, DI...",31,0
9,356434,"ORGANIC GRAIN VINEGAR, WATER, ORGANIC MUSTARD ...",7,5


In [20]:
csv_file = "../Resources/food.csv"
food_df = pd.read_csv(csv_file)
food_df.head()

,fdc_id,data_type,description,food_category_id,publication_date
0,356425,branded_food,MOCHI ICE CREAM BONBONS,NaN,2019-04-01
1,356426,branded_food,CHIPOTLE BARBECUE SAUCE,NaN,2019-04-01
2,356427,branded_food,HOT & SPICY BARBECUE SAUCE,NaN,2019-04-01
3,356428,branded_food,BARBECUE SAUCE,NaN,2019-04-01
4,356429,branded_food,BARBECUE SAUCE,NaN,2019-04-01


In [21]:
#Re-organize the food dataframe to include only what we need
clean_food_df = food_df[['fdc_id', 'description']].copy()
clean_food_df.head()

,fdc_id,description
0,356425,MOCHI ICE CREAM BONBONS
1,356426,CHIPOTLE BARBECUE SAUCE
2,356427,HOT & SPICY BARBECUE SAUCE
3,356428,BARBECUE SAUCE
4,356429,BARBECUE SAUCE


In [22]:
clean_branded_food_df.head()

,fdc_id,brand_owner,ingredients,branded_food_category
0,356425,"G. T. Japan, Inc.","ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR...",Ice Cream & Frozen Yogurt
1,356426,FRESH & EASY,"WATER, SUGAR, TOMATO PASTE, MOLASSES, DISTILLE...","Ketchup, Mustard, BBQ & Cheese Sauce"
2,356427,FRESH & EASY,"SUGAR, WATER, DISTILLED VINEGAR, TOMATO PASTE,...","Ketchup, Mustard, BBQ & Cheese Sauce"
3,356428,FRESH & EASY,"TOMATO PUREE (WATER, TOMATO PASTE), SUGAR, DIS...","Ketchup, Mustard, BBQ & Cheese Sauce"
4,356429,FRESH & EASY,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PASTE,...","Ketchup, Mustard, BBQ & Cheese Sauce"


In [23]:
#Set index to id
indexed_df = clean_branded_food_df.set_index('fdc_id')
indexed_df.head()

,brand_owner,ingredients,branded_food_category
fdc_id,,,
356425,"G. T. Japan, Inc.","ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR...",Ice Cream & Frozen Yogurt
356426,FRESH & EASY,"WATER, SUGAR, TOMATO PASTE, MOLASSES, DISTILLE...","Ketchup, Mustard, BBQ & Cheese Sauce"
356427,FRESH & EASY,"SUGAR, WATER, DISTILLED VINEGAR, TOMATO PASTE,...","Ketchup, Mustard, BBQ & Cheese Sauce"
356428,FRESH & EASY,"TOMATO PUREE (WATER, TOMATO PASTE), SUGAR, DIS...","Ketchup, Mustard, BBQ & Cheese Sauce"
356429,FRESH & EASY,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PASTE,...","Ketchup, Mustard, BBQ & Cheese Sauce"


In [24]:
#Split the many ingredients in ingredients column into separate columns
ingredients_split = indexed_df['ingredients'].str.split(', ', -1, expand=True)
ingredients_split.head()

,0,1,2,3,4,5,6,7,8,9,...,163,164,165,166,167,168,169,170,171,172
fdc_id,,,,,,,,,,,,,,,,,,,,,
356425,ICE CREAM INGREDIENTS: MILK,CREAM,SUGAR,STRAWBERRIES (STRAWBERRIES,SUGAR),CORN SYRUP SOLIDS,SKIM MILK,WHEY,NATURAL FLAVOR,GUAR GUM,...,None,None,None,None,None,None,None,None,None,None
356426,WATER,SUGAR,TOMATO PASTE,MOLASSES,DISTILLED VINEGAR,CONTAINS 2% OR LESS OF: CORN STARCH,SALT,DRIED CHIPOTLE PEPPER,NATURAL SMOKE FLAVOR,MUSTARD FLOUR,...,None,None,None,None,None,None,None,None,None,None
356427,SUGAR,WATER,DISTILLED VINEGAR,TOMATO PASTE,MOLASSES,MODIFIED CORN STARCH,SALT,PINEAPPLE JUICE CONCENTRATE,CONTAINS 1% OR LESS OF: JALAPENO PEPPERS,SPICE,...,None,None,None,None,None,None,None,None,None,None
356428,TOMATO PUREE (WATER,TOMATO PASTE),SUGAR,DISTILLED VINEGAR,MOLASSES,WATER,MODIFIED CORN STARCH,SALT,BOURBON WHISKEY,CONTAINS 1% OR LESS OF: MUSTARD FLOUR,...,None,None,None,None,None,None,None,None,None,None
356429,SUGAR,DISTILLED VINEGAR,WATER,TOMATO PASTE,HONEY,MOLASSES,MODIFIED CORN STARCH,SALT,CONTAINS 1% OR LESS OF: NATURAL FLAVOR,PINEAPPLE JUICE CONCENTRATE,...,None,None,None,None,None,None,None,None,None,None


In [25]:
#Merge two dataframes together
ingredients_df = pd.merge(indexed_df, ingredients_split, on='fdc_id', how='inner')
ingredients_df.head()

,brand_owner,ingredients,branded_food_category,0,1,2,3,4,5,6,...,163,164,165,166,167,168,169,170,171,172
fdc_id,,,,,,,,,,,,,,,,,,,,,
356425,"G. T. Japan, Inc.","ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR...",Ice Cream & Frozen Yogurt,ICE CREAM INGREDIENTS: MILK,CREAM,SUGAR,STRAWBERRIES (STRAWBERRIES,SUGAR),CORN SYRUP SOLIDS,SKIM MILK,...,None,None,None,None,None,None,None,None,None,None
356426,FRESH & EASY,"WATER, SUGAR, TOMATO PASTE, MOLASSES, DISTILLE...","Ketchup, Mustard, BBQ & Cheese Sauce",WATER,SUGAR,TOMATO PASTE,MOLASSES,DISTILLED VINEGAR,CONTAINS 2% OR LESS OF: CORN STARCH,SALT,...,None,None,None,None,None,None,None,None,None,None
356427,FRESH & EASY,"SUGAR, WATER, DISTILLED VINEGAR, TOMATO PASTE,...","Ketchup, Mustard, BBQ & Cheese Sauce",SUGAR,WATER,DISTILLED VINEGAR,TOMATO PASTE,MOLASSES,MODIFIED CORN STARCH,SALT,...,None,None,None,None,None,None,None,None,None,None
356428,FRESH & EASY,"TOMATO PUREE (WATER, TOMATO PASTE), SUGAR, DIS...","Ketchup, Mustard, BBQ & Cheese Sauce",TOMATO PUREE (WATER,TOMATO PASTE),SUGAR,DISTILLED VINEGAR,MOLASSES,WATER,MODIFIED CORN STARCH,...,None,None,None,None,None,None,None,None,None,None
356429,FRESH & EASY,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PASTE,...","Ketchup, Mustard, BBQ & Cheese Sauce",SUGAR,DISTILLED VINEGAR,WATER,TOMATO PASTE,HONEY,MOLASSES,MODIFIED CORN STARCH,...,None,None,None,None,None,None,None,None,None,None


In [26]:
#Drop original ingredients column
ingredients_df = ingredients_df.drop(labels = ['ingredients'], axis=1)
ingredients_df.head()

,brand_owner,branded_food_category,0,1,2,3,4,5,6,7,...,163,164,165,166,167,168,169,170,171,172
fdc_id,,,,,,,,,,,,,,,,,,,,,
356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,ICE CREAM INGREDIENTS: MILK,CREAM,SUGAR,STRAWBERRIES (STRAWBERRIES,SUGAR),CORN SYRUP SOLIDS,SKIM MILK,WHEY,...,None,None,None,None,None,None,None,None,None,None
356426,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",WATER,SUGAR,TOMATO PASTE,MOLASSES,DISTILLED VINEGAR,CONTAINS 2% OR LESS OF: CORN STARCH,SALT,DRIED CHIPOTLE PEPPER,...,None,None,None,None,None,None,None,None,None,None
356427,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",SUGAR,WATER,DISTILLED VINEGAR,TOMATO PASTE,MOLASSES,MODIFIED CORN STARCH,SALT,PINEAPPLE JUICE CONCENTRATE,...,None,None,None,None,None,None,None,None,None,None
356428,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",TOMATO PUREE (WATER,TOMATO PASTE),SUGAR,DISTILLED VINEGAR,MOLASSES,WATER,MODIFIED CORN STARCH,SALT,...,None,None,None,None,None,None,None,None,None,None
356429,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",SUGAR,DISTILLED VINEGAR,WATER,TOMATO PASTE,HONEY,MOLASSES,MODIFIED CORN STARCH,SALT,...,None,None,None,None,None,None,None,None,None,None


In [27]:
#Reset index to make id a column again
ingredients_df = ingredients_df.reset_index()
ingredients_df.head()

,fdc_id,brand_owner,branded_food_category,0,1,2,3,4,5,6,...,163,164,165,166,167,168,169,170,171,172
0,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,ICE CREAM INGREDIENTS: MILK,CREAM,SUGAR,STRAWBERRIES (STRAWBERRIES,SUGAR),CORN SYRUP SOLIDS,SKIM MILK,...,None,None,None,None,None,None,None,None,None,None
1,356426,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",WATER,SUGAR,TOMATO PASTE,MOLASSES,DISTILLED VINEGAR,CONTAINS 2% OR LESS OF: CORN STARCH,SALT,...,None,None,None,None,None,None,None,None,None,None
2,356427,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",SUGAR,WATER,DISTILLED VINEGAR,TOMATO PASTE,MOLASSES,MODIFIED CORN STARCH,SALT,...,None,None,None,None,None,None,None,None,None,None
3,356428,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",TOMATO PUREE (WATER,TOMATO PASTE),SUGAR,DISTILLED VINEGAR,MOLASSES,WATER,MODIFIED CORN STARCH,...,None,None,None,None,None,None,None,None,None,None
4,356429,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",SUGAR,DISTILLED VINEGAR,WATER,TOMATO PASTE,HONEY,MOLASSES,MODIFIED CORN STARCH,...,None,None,None,None,None,None,None,None,None,None


In [28]:
separated_ingredient_df = pd.melt(ingredients_df, id_vars = ['fdc_id', 'brand_owner', 'branded_food_category'], value_name = 'ingredient')
separated_ingredient_df.head(25)

,fdc_id,brand_owner,branded_food_category,variable,ingredient
0,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,0,ICE CREAM INGREDIENTS: MILK
1,356426,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",0,WATER
2,356427,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",0,SUGAR
3,356428,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",0,TOMATO PUREE (WATER
4,356429,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",0,SUGAR
5,356430,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",0,SUGAR
6,356431,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",0,SUGAR
7,356432,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",0,WATER
8,356433,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",0,TOMATO PUREE
9,356434,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",0,ORGANIC GRAIN VINEGAR


In [29]:
#Drop un-necessary column
separated_ingredient_df = separated_ingredient_df.drop(labels = ['variable'], axis=1)
separated_ingredient_df.head()

,fdc_id,brand_owner,branded_food_category,ingredient
0,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,ICE CREAM INGREDIENTS: MILK
1,356426,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",WATER
2,356427,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",SUGAR
3,356428,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",TOMATO PUREE (WATER
4,356429,FRESH & EASY,"Ketchup, Mustard, BBQ & Cheese Sauce",SUGAR


In [30]:
# Check the data
separated_ingredient_df.count()

fdc_id                   45044010
brand_owner              44901631
branded_food_category    43293596
ingredient                3880097
dtype: int64

In [31]:
# Drop null values
separated_ingredient_df = separated_ingredient_df.dropna()
separated_ingredient_df.count()

fdc_id                   3693889
brand_owner              3693889
branded_food_category    3693889
ingredient               3693889
dtype: int64

In [32]:
#Merge two dataframes into one on 'fdc_id'
#DATASETS ARE MERGED BEFORE INSERTING INTO DB
combined_df = pd.merge(separated_ingredient_df, clean_food_df, on="fdc_id")
combined_df.head()

,fdc_id,brand_owner,branded_food_category,ingredient,description
0,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,ICE CREAM INGREDIENTS: MILK,MOCHI ICE CREAM BONBONS
1,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,CREAM,MOCHI ICE CREAM BONBONS
2,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,SUGAR,MOCHI ICE CREAM BONBONS
3,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,STRAWBERRIES (STRAWBERRIES,MOCHI ICE CREAM BONBONS
4,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,SUGAR),MOCHI ICE CREAM BONBONS


In [33]:
#Check count of rows
combined_df.count()

fdc_id                   3693889
brand_owner              3693889
branded_food_category    3693889
ingredient               3693889
description              3693889
dtype: int64

In [34]:
#Connect to local database
database_path = "organic_food_db.sqlite"
engine = create_engine(f'sqlite:///{database_path}')

In [35]:
#Check for tables
engine.table_names()

[]

In [36]:
#Use pandas to load csv converted dataframe into database
combined_df.to_sql(name='tbl_combined_food', con=engine, if_exists='append', index=False)
ingred_df.to_sql(name='tbl_organic_count', con=engine, if_exists='append', index=False)

In [37]:
engine.table_names()

['tbl_combined_food', 'tbl_organic_count']

In [38]:
#Confirm data has been added by querying the table
pd.read_sql_query('select * from tbl_combined_food', con=engine).head()

,fdc_id,brand_owner,branded_food_category,ingredient,description
0,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,ICE CREAM INGREDIENTS: MILK,MOCHI ICE CREAM BONBONS
1,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,CREAM,MOCHI ICE CREAM BONBONS
2,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,SUGAR,MOCHI ICE CREAM BONBONS
3,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,STRAWBERRIES (STRAWBERRIES,MOCHI ICE CREAM BONBONS
4,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,SUGAR),MOCHI ICE CREAM BONBONS


In [39]:
pd.read_sql_query('select * from tbl_organic_count', con=engine).head()

,fdc_id,ingredients,ingredient_count,organic_count
0,356425,"ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR...",21,0
1,356426,"WATER, SUGAR, TOMATO PASTE, MOLASSES, DISTILLE...",13,0
2,356427,"SUGAR, WATER, DISTILLED VINEGAR, TOMATO PASTE,...",15,0
3,356428,"TOMATO PUREE (WATER, TOMATO PASTE), SUGAR, DIS...",16,0
4,356429,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PASTE,...",15,0


In [40]:
#SECOND JOIN/MERGE OF DATASETS
pd.read_sql_query('select * from tbl_combined_food a join tbl_organic_count b on a.fdc_id = b.fdc_id', con=engine).head()

,fdc_id,brand_owner,branded_food_category,ingredient,description,fdc_id,ingredients,ingredient_count,organic_count
0,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,ICE CREAM INGREDIENTS: MILK,MOCHI ICE CREAM BONBONS,356425,"ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR...",21,0
1,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,CREAM,MOCHI ICE CREAM BONBONS,356425,"ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR...",21,0
2,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,SUGAR,MOCHI ICE CREAM BONBONS,356425,"ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR...",21,0
3,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,STRAWBERRIES (STRAWBERRIES,MOCHI ICE CREAM BONBONS,356425,"ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR...",21,0
4,356425,"G. T. Japan, Inc.",Ice Cream & Frozen Yogurt,SUGAR),MOCHI ICE CREAM BONBONS,356425,"ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR...",21,0


In [43]:
pd.read_sql_query('select fdc_id, ingredient_count from tbl_organic_count where organic_count>0', con=engine).head()

,fdc_id,ingredient_count
0,356434,7
1,356437,6
2,356441,7
3,356444,13
4,356468,12
